In [ ]:
import cv2
import numpy as np
import os

# Function to load multiple template images from a directory
def load_templates(directory):
    templates = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpeg") or filename.endswith(".jpg"):
            template_path = os.path.join(directory, filename)
            template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
            if isinstance(template, np.ndarray):  # Check if template is a NumPy array
                templates.append(template)
            else:
                print(f"Warning: Invalid template found: {filename}")
    return templates

# Function to track players and count team members
def track_players(video_path, cascade_path):
    # Load the Haar cascade classifier
    face_cascade = cv2.CascadeClassifier(cascade_path)

    # Open video file
    cap = cv2.VideoCapture(video_path)

    # Initialize player count for each team
    team1_count = 0
    team2_count = 0

    # Define HSV color ranges for yellow and red jerseys
    lower_yellow = np.array([8, 60, 110])  # Lower HSV range for yellow color
    upper_yellow = np.array([35, 255, 255])  # Upper HSV range for yellow color

    lower_red = np.array([0, 160, 170])  # Lower HSV range for red color
    upper_red = np.array([14, 255, 255])  # Upper HSV range for red color

    # Loop through video frames
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to HSV color space
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Detect players using the face cascade
        players = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Draw rectangles around detected players and count players for each team
        for (x, y, w, h) in players:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Extract the region of interest (ROI) in HSV color space
            roi_hsv = hsv_frame[y:y+h, x:x+w]

            # Check if the mean HSV values in the ROI fall within the yellow or red color range
            if np.mean(roi_hsv[:, :, 0]) >= lower_yellow[0] and np.mean(roi_hsv[:, :, 0]) <= upper_yellow[0]:
                team1_count += 1
            elif np.mean(roi_hsv[:, :, 0]) >= lower_red[0] and np.mean(roi_hsv[:, :, 0]) <= upper_red[0]:
                team2_count += 1

        # Display frame with player rectangles and player counts
        cv2.putText(frame, f'Team 1: {team1_count}', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f'Team 2: {team2_count}', (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.imshow('Volleyball Tracking', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    # Release video capture and close windows
    cap.release()
    cv2.destroyAllWindows()

# Load multiple template images from a directory
template_directory = "/Users/rohanshenoy/Desktop/python/ball_templates"
templates = load_templates(template_directory)

# Load the volleyball match video
video_path = "/Users/rohanshenoy/Downloads/volleyball_match.mp4"

# Create tracker
tracker = cv2.TrackerMIL_create()

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video = cv2.VideoCapture(video_path)
output_video = cv2.VideoWriter('output.avi', fourcc, 30.0, (int(video.get(3)), int(video.get(4))))

# Initialize dictionary to store template trackers
template_trackers = {}

# Define Haar cascade path for player detection
cascade_path = '/Users/rohanshenoy/Desktop/python/haarcascade_fullbody.xml'

# Track players in the volleyball match video
track_players(video_path, cascade_path)

# Open video file for volleyball tracking
video = cv2.VideoCapture(video_path)

# Loop through video frames for ball tracking
while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    # Resize the frame to a smaller size for better computation
    resized_frame = cv2.resize(frame, None, fx=0.5, fy=0.5)  # Adjust the scaling factor as needed

    # Convert resized frame to grayscale
    gray = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

    for template in templates:
        # Resize the template to match the resized frame
        resized_template = cv2.resize(template, None, fx=0.5, fy=0.5)  # Adjust the scaling factor as needed

        # Perform template matching with the current template image
        res = cv2.matchTemplate(gray, resized_template, cv2.TM_CCOEFF_NORMED)

        # Define a threshold for the match score
        threshold = 0.77

        # Get the location of the match
        loc = np.where(res >= threshold)

        # Draw a rectangle around the matched region
        for pt in zip(*loc[::-1]):
            cv2.rectangle(resized_frame, pt, (pt[0] + resized_template.shape[1], pt[1] + resized_template.shape[0]), (0, 255, 0), 2)

            # Convert template shape (tuple) to string for dictionary key
            template_key = str(template.shape)

            # Initialize tracker for the template if not already initialized
            if template_key not in template_trackers:
                tracker = cv2.TrackerMIL_create()
                template_trackers[template_key] = tracker
                tracker.init(frame, (pt[0], pt[1], resized_template.shape[1], resized_template.shape[0]))

            # Update tracker and draw trajectory
            ok, bbox = template_trackers[template_key].update(frame)
            if ok:
                p1 = (int(bbox[0]), int(bbox[1]))
                p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
                cv2.rectangle(frame, p1, p2, (0, 255, 0), 2)

    # Write the frame to the output video
    output_video.write(resized_frame)

    # Display the frame
    cv2.imshow('Volleyball Tracking', resized_frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the video objects
video.release()
output_video.release()

# Close all OpenCV windows
cv2.destroyAllWindows()
